# 树状数组

树状数组(Binary Indexed Tree, or Fenwick Tree)是一种维护数组前缀和、区间和的数据结构<br>
思想和跳表有点类似:<br>
·跳表:添加索引，高效维护链表<br>
·树状数组:添加索引，高效维护数组<br>

## 树状数组的思想

### 分块

分块的本质还是分治，即对原始数据进行适当划分，并在划分后的每一个快上预处理部分信息，从而较暴力算法取得更优的时间复杂度。与树状数组类似的还有sqrt分解。

### 倍增

倍增，字面意思就是“成倍增长”。在进行递推时，如果状态空间很大，线性递推无法满足时间与空间复杂度的要求，可以通过成倍增长的方法，只递推状态空间中在2的整数次幂位置上的值作为代表；而当需要其他位置上的值时，可以通过“任意整数可以表示成若干个2的次幂项的和”这一性质，使用之前求出的代表值拼成所需的值。

#### 241. 楼兰图腾 https://www.acwing.com/problem/content/243/

In [ ]:
N=200000 
class Tree:
    def __init__(self,n):
        self.tr=[0]*(n+1)
        self.n=n
    def lowbit(self,x):
        return x&(-x)
    def add(self,i,v):
        while i<=self.n:
            self.tr[i]+=v
            i+=self.lowbit(i)
    def getpr(self,i):
        res=0
        while i:
            res+=self.tr[i]
            i-=self.lowbit(i)
        return res

n=int(input())
q=[0]*(n+1)
front_larger=[0]*(n+1)
front_less=[0]*(n+1)
back_larger=[0]*(n+1)
back_less=[0]*(n+1)

q[1:]=list(map(int,input().split()))

tr1=Tree(N+1)
for i in range(1,n+1):
    front_larger[i]=tr1.getpr(N)-tr1.getpr(q[i]-1)
    front_less[i]=tr1.getpr(q[i]-1)
    tr1.add(q[i],1)

tr2=Tree(N+1)
for i in range(n,0,-1):
    back_larger[i]=tr2.getpr(N)-tr2.getpr(q[i]-1)
    back_less[i]=tr2.getpr(q[i]-1)
    tr2.add(q[i],1)

up_res,down_res=0,0
for i in range(1,n+1):
    up_res+=(front_less[i]*back_less[i])
    down_res+=(front_larger[i]*back_larger[i])

print(down_res,up_res)

#### 242. 一个简单的整数问题 https://www.acwing.com/problem/content/248/

In [ ]:
class NumArray:
    def __init__(self, nums):
        nums2=nums.copy()
        self.n=len(nums)+1
        self.tree=[0]*(self.n)
        for i in range(1,self.n):
            if i!=len(nums):
                nums2[i]=nums[i]-nums[i-1]
            self.tree[i]=sum(nums2[i-self.getkbit(i):i])

    def getkbit(self,k):
        return k&-k
    
    def add(self, index: int, val: int):
        while index<self.n:
            self.tree[index]+=val
            index+=self.getkbit(index)

    def getprefix(self,n):
        res=0
        while n:
            res+=self.tree[n]
            n-=self.getkbit(n)
        return res
a=input()
n,row=[int(i) for i in a.split()]
b=input()
arr=[int(i) for i in b.split()]
tree=NumArray(arr)
for i in range(row):
    c=input()
    op=c.split()[0]
    if op=="Q":
        idx=int(c.split()[1])
        print(tree.getprefix(idx))
    else:
        left=int(c.split()[1])
        right=int(c.split()[2])
        val=int(c.split()[3])
        tree.add(left,val)
        tree.add(right+1,-val)

## 树状数组的局限性

树状数组有实现简单、效率高、省空间等诸多优势,但也有很大的局限性<br>
维护的信息需要满足区间减法性质<br>
·不然无法通过前缀和相减得到区间和<br>
·例如无法直接拿来维护区间最值<br>
不能很好地支持修改操作<br>
·单点修改需要先求出差值，转化为增加操作<br>
·基本上难以支持区间修改（修改连续的一段数据)<br>

# 线段树

线段树(Segment Tree)是一种基于分治思想的二叉树结构，用于在区间上进行信息统计。<br>
·线段树的每个节点都代表一个闭区间。<br>
·线段树具有唯一的根节点，代表的区间是整个统计范围，如[1,N]。<br>
·线段树的每个叶节点都代表一个长度为1的元区间[x, c]。<br>
对于每个内部节点[1,r]，它的左子节点是[l,mid]，右子节点是[mid + 1,r]，其中mid = (l+r)/2(向下取整)<br>

## 区间修改与延迟标记


区间查询与修改的区别:<br>
区间查询遇到完全覆盖的区间，可以直接返回，最多处理2logn个结点<br>
对于完全覆盖的区间，区间修改会影响它的所有子结点，时间复杂度最坏O(n)<br>


解决方案:懒惰标记（又称延迟标记)<br>
·回想二叉堆的懒惰删除<br>
·遇到完全覆盖的区间，先打一个修改标记，只要不到子结点中查询，就不往下继续修改<br>
·在以后的递归查询/修改中遇到了标记，再往下传递一层<br>

时间复杂度优化到O(log(n))<br>
思想:我mark一下这个bug要改，你们谁以后看见了记得先改一下再run哈，碰不到就不管了<br>

# 位运算

机器采用二进制对数值进行表示、存储和运算<br>
在程序中恰当地使用二进制，可以提高运行效率<br>

## 异或运算（xor)

相同为0，不同为1<br>
也可用“不进位加法”来理解<br>
异或运算的特点与应用:<br>
x ^0=x<br>
x ^x=0<br>
结合律: a ^ b ^ c=a ^ (b ^ c)= (a ^ b) ^ c<br>
成对变换:x^1 (0^1=1,1^1=0; 2^1=3，3^1=2;4^1=5，5^1=4)<br>
交换两个数: a <- a ^ b, b <- a ^ b, a <- a ^ b<br>

## 指定位置的位运算

二进制数的最右边为第0位<br>
获取×在二进制下的第n位（0或者1) :(x >>n)& 1<br>
将×在二进制下的第n位置为1:x|(1<<n)<br>
将×在二进制下的第n位置为O: x &(~(1<<n))<br>
将x在二进制下的第n位取反:x^(1<<n)<br>
将×最右边的n位清零:x & (~0<<n)<br>
将×最高位至第n位（含）清零:x& ((1 <<n)- 1)<br>

## 位运算实战要点

判断奇偶:<br>
· x % 2 == 1 →(x & 1)== 1<br>
· x % 2 ==0 →(x & 1)== 0<br>

除以2的幂次︰<br>
. x/2 →x >> 1<br>
.mid =(left + right)/ 2; mid = (left + right) >> 1;<br>

lowbit:<br>
·得到最低位的1: x & -x或x &(~x+ 1)<br>
·清零最低位的1: x =x &(x-1)<br>

#### 89. a^b https://www.acwing.com/problem/content/91/

In [ ]:
a,b,p=map(int,input().split())
res=1%p
while b:
    if b&1:
        res=res*a%p
    a=a*a%p
    b>>=1
print(res)

#### 90. 64位整数乘法 https://www.acwing.com/problem/content/92/

In [ ]:
a=int(input())
b=int(input())
p=int(input())
res=0%p
while b:
    if b&1:
        res=(res+a)%p
    a=a*2%p
    b>>=1
print(res)

#### 91. 最短Hamilton路径 https://www.acwing.com/problem/content/93/

In [ ]:
n=int(input())
weight=[]
for i in range(n):
    a=input()
    temp=[int(i) for i in a.split()]
    weight.append(temp)
m=1<<n
dp=[[float("inf")]*(m) for _ in range(n)]
dp[0][1]=0
for i in range(m):
    for j in range(n):
        if (i>>j)&1:
            for k in range(n):
                if (i-(1<<j))>>k&1:
                    dp[j][i]=min(dp[j][i],dp[k][i-(1<<j)]+weight[k][j])
print(dp[n-1][m-1])

# 期末串讲&总复习

## 高频主题

计算机科学基础:数据结构、树/图、DFS/BFS

状态空间与子问题划分<br>
·递归、分治、动态规划的设计

决策与候选集合优化<br>
.“滑动窗口”系列的多种维护方法，固定一端+移动一端+消除冗余的思想

#### *单调栈*

#### 131. 直方图中最大的矩形 https://www.acwing.com/problem/content/133/

In [ ]:
def getarea(lst):
    ans=0
    lst=[0]+lst+[0]
    stack=[]
    for key,val in enumerate(lst):
        if key==0:
            stack.append(key)
        elif val>=lst[stack[-1]]:
            stack.append(key)
        elif val<lst[stack[-1]]:
            while val<lst[stack[-1]]:
                h=lst[stack.pop()]
                w=key-stack[-1]-1
                ans=max(ans,w*h)
            stack.append(key)
    return ans
while True:
    a=input()
    if a=="0":
        break
    lst=list(map(int,a.split()))
    print(getarea(lst[1:]))

#### *单调队列*

#### 135. 最大子序和 https://www.acwing.com/problem/content/description/137/

In [ ]:
n,m=map(int,input().split())
lst=list(map(int,input().split()))
arr=[0]*(n+1)
for i in range(n):
    arr[i+1]=arr[i]+lst[i]
queue=[0]
ans=-float("inf")
for i in range(1,n+1):
    while queue and queue[0]<i-m:
        queue.pop(0)
    
    ans=max(ans,arr[i]-arr[queue[0]])
    while queue and arr[queue[-1]]>arr[i]:
        queue.pop()
    queue.append(i)
print(ans)

#### 154. 滑动窗口 https://www.acwing.com/problem/content/description/156/

In [ ]:
n,k=map(int,input().split())
lst=list(map(int,input().split()))
maxqueue=[]
minqueue=[]
maxans=[]
minans=[]
for i in range(n):
    while maxqueue and maxqueue[0]<=i-k:
        maxqueue.pop(0)
    while minqueue and minqueue[0]<=i-k:
        minqueue.pop(0)
    while maxqueue and lst[maxqueue[-1]]<lst[i]:
        maxqueue.pop()
    while minqueue and lst[minqueue[-1]]>lst[i]:
        minqueue.pop()
    maxqueue.append(i)
    minqueue.append(i)
    if i>=k-1:
        maxans.append(lst[maxqueue[0]])
        minans.append(lst[minqueue[0]])
print(" ".join([str(i) for i in minans]))
print(" ".join([str(i) for i in maxans]))

## 高端技巧

Pre-calculation:空间换时间（前缀和、差分等)

二分答案:求解转化为判定<br>
二分的基础用法是在单调序列或单调函数中进行查找；因此当问题的答案具有单调性时，就可以通过二分把求解转化为判定。有点类似于数学里碰到一个很难的问题，拿几个值去套答案比直接求解要来得简单，而我们还可以借住计算机强大的算力更快捷地套答案。

离线批处理，关键事件思想

#### *前缀和*

#### 99. 激光炸弹 https://www.acwing.com/problem/content/101/

In [ ]:
N,R=map(int,input().split())
arr=[[0] * 5010 for _ in range(5010)]
xmin=float("inf")
ymin=float("inf")
xmax=0
ymax=0
for i in range(N):
    x,y,v=map(int,input().split())
    arr[y+1][x+1]+=v
    xmin=min(xmin,x)
    ymin=min(ymin,y)
    xmax=max(xmax,x)
    ymax=max(ymax,y)
for y in range(ymin+1,ymax+2):
    for x in range(xmin+1,xmax+2):
        arr[y][x]+=arr[y-1][x]+arr[y][x-1]-arr[y-1][x-1]
res=0
for y in range(ymin+1,ymax+2):
    for x in range(xmin+1,xmax+2):
        ry,rx=min(ymax+1,y+R-1),min(xmax+1,x+R-1)
        res=max(res,arr[ry][rx]+arr[y-1][x-1]-arr[ry][x-1]-arr[y-1][rx])
print(res)

#### *差分*

#### 101. 最高的牛 https://www.acwing.com/problem/content/103/

In [ ]:
N,P,H,M =map(int,input().split())
height=[0]*N
s=set()
for i in range(M):
    left,right=map(int,input().split())
    if left>right:
        left,right=right,left
    if (left,right) not in s:
        s.add((left,right))
        if left!=right:
            height[left]-=1
            height[right-1]+=1
res=[H+height[0]]
for i in range(1,N):
    height[i]=height[i]+height[i-1]
    res.append(H+height[i])
for i in res:
    print(i)

#### *二分*

#### 789. 数的范围 https://www.acwing.com/problem/content/description/791/

In [ ]:
n,m=map(int,input().split())
lst=list(map(int,input().split()))
def find(arr,target,n):
    left=0
    right=n
    ans=[]
    while left<right:
        mid=(left+right)>>1
        if arr[mid]>=target:
            right=mid
        else:
            left=mid+1
    ans.append(left)
    left=-1
    right=n-1
    while left<right:
        mid=(left+right+1)>>1
        if arr[mid]<=target:
            left=mid
        else:
            right=mid-1
    ans.append(left)
    if ans[0]>ans[1]:
        return [-1,-1]
    return ans
for i in range(m):
    t=int(input())
    print(" ".join([str(i) for i in find(lst,t,n)]))

# 课程总结

## 三条主线

线性结构线：数组、链表-->栈、队列-->前缀和、差分-->双指针、滑动窗-->哈希、集合、映射-->字符串<br>
树形结构线：递归、分治-->树、图-->搜索-->堆、二叉搜索树-->字典树、并查集-->图论算法<br>
状态空间先：（搜索）-->动态规划、贪心-->图论算法

## 两个副本


顺序优化副本：排序、二分<br>
高级数据结构副本：平衡树、跳表、树状数组、线段树

#### 排序的离散化

排序的第一个应用是离散化，就是把无穷大集合中的若干元素映射为有限集合以便于统计的方法